## Frontend Flask Server Implementation

### Loading Libraries 


In [2]:
from flask import Flask, request, render_template, jsonify
from tinydb import TinyDB, Query, where
from tinydb.operations import set
import os
import subprocess
import pyttsx3
import re

ModuleNotFoundError: No module named 'tinydb'

### Model of  Frontend

![Alt](https://github.com/vishalprabha/Graphic-based-verbal-reasoning/raw/master/front_back_end.JPG)

### DB Initialization and Constants

Initialization of TinyDB by passing the path of the data file, and setting up other path constants

In [1]:
''' Constants  '''
IMAGE_URL = '..\\static\\test_images\\'
IMAGE_PATH = 'web\\static\\test_images\\'

''' Text to speech Init '''


''' DB COnfiguration '''
db = TinyDB('db/db.json')
db_query = Query()


NameError: name 'TinyDB' is not defined

### Routes or Endpoints of Flask Server

Various endpoints are defined in the server to handle the incoming data and process it. We have defined majorly 3 routes or endpoints in the server
1. Index Route: It handles the first render of the html page and sets the default data

In [ ]:
@app.route('/', methods=["GET","POST"])
def index():
    init_data = db.all()
    print(init_data)
    return render_template("index.html", data = init_data[0])

2. Conversation Route: It handles the conversation between the user and GBVR Asst, and takes in the question and the image provided and creates the asynchronous subprocess call to the backend, and the result is returned.

In [ ]:
@app.route('/convo', methods= ['GET', 'POST'])
def converstion():
    if request.method == 'GET':
        question = request.args['msg']
        print("MESSAGE : " + question)

        # subprocess call to gbvr by mmoving out to root
        # print("current working dir prev " + os.getcwd())
        web_dir = os.getcwd()
        os.chdir("..\\")
        # print("current working dir after " + os.getcwd())

        img_path = db.search(db_query.chat_id == 1)[0]['img_path']
        print(img_path)

        std_out = open("web\\output.txt", "r+")
        # std_out.write(question)
        s = subprocess.Popen(['python', 'demo.py', '-image_file_name', img_path, '-question', question], stdout = std_out)
        s.wait()
        os.chdir(web_dir)

        # parse the output to dict
        result_dict = parse_output()
        db.update(set("q_a", [{"question": question, "result": result_dict}]), db_query.chat_id == 1)

        return jsonify(result_dict)

3. Upload Route: It handles the upload of the images to the TinyDB

In [ ]:
@app.route('/upload', methods= ['GET', 'POST'])
def upload_file():
    if request.method == 'GET':
        print("Hi upload POST call", request.args)

        file_name = os.path.basename(request.args['file'])
        file_url = IMAGE_URL + file_name
        file_path = IMAGE_PATH + file_name

        db.update(set("img_url", file_url), db_query.chat_id == 1)
        db.update(set("img_path", file_path), db_query.chat_id == 1 )
        return file_url
    else:
        return '404'

### Text to Speech Engine

The TTS route is defined for converting the text to speech, this is fired on the retrival of the result. This route handles the speech engine properties and the result text to be read out.

In [ ]:
@app.route('/tts', methods= ['GET', "POST"])
def tts():
    if request.method == 'GET':
        message = request.args['message']
        print("TTS : " + message)
        message = re.sub(r'<br/>', "", message)
        # message.replace("<br/>", "")
        print("TTS : " + message)
        engine = pyttsx3.init()
        voices = engine.getProperty('voices')
        rate = engine.getProperty('rate')
        engine.setProperty('rate', rate-50) #set rate
        volume = engine.getProperty('volume')
        engine.setProperty('volume', volume+0.50) #set volume
        voices = engine.getProperty('voices')
        engine.setProperty('voice', 'english+f4') #voice id 0
        engine.say(message)
        engine.runAndWait()
        return ''